In [1]:
pip install -q yahoo_fin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.0 MB/s eta 0:00:00


In [ ]:
pip install seaborn

In [ ]:
import seaborn as sns

In [1]:
!mkdir -p data results logs


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !cp  /content/drive/MyDrive/colab/results/* ./results/
else:
  ! cp ~/Google\ Drive/My\ Drive/colab/results/* ./results/

Todos:

 

1.   Load previously saved model before training
2.   Allow eval only without test/train spliting
3.   Incremental data load
4.    Add S&P, QQQ etc to the model
5. Save model to google drive






Run following in local Jupyter

In [3]:
import os
import sys
path = os.path.expanduser('~/src/runml')
print(path)
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata

/Users/ahsank/src/runml


Run following 2 cells for colab to import libraries

In [5]:
!git clone https://github.com/ahsank/runml

Cloning into 'runml'...
remote: Enumerating objects: 634, done.
remote: Counting objects: 100% (167/167), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 634 (delta 110), reused 35 (delta 27), pack-reused 467
Receiving objects: 100% (634/634), 4.17 MiB | 15.59 MiB/s, done.
Resolving deltas: 100% (398/398), done.


In [6]:
import importlib
import sys
path = 'runml'
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata


Run following if the library code is modified locally

In [131]:
from importlib import reload
reload(findata)
reload(pipeline)

<module 'runml.pipeline' from '/Users/ahsank/src/runml/runml/pipeline.py'>

In [135]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [132]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
pending = ['PAR']
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRCT', 'CRWD',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DXCM',
           'ESTC', 'ETSY', 'EXPE', 'FOUR', 'GFS', 'GLBE', 'GOGO',
           'INDI', 'INMD', 'INTA', 'IOT','JKS', 'LUMN', 'LYFT',
           'MAXR', 'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
           'MTCH',  'NVCR','OKTA', 'OLED', 'OSTK',
           'PANW', 'PAYO', 'PD', 'PLUG', 'PI', 'PINS', 'PTON', 'PUBM', 'RBLX',
           'SNAP', 'SNOW', 'SPLK', 'SPOT','STEM', 'STNE','SWAV',
           'TEAM', 'TDOC', 'TOST', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))
#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='low', trading=pipeline.LowTrading)[0]
#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='high', trading=pipeline.HighTrading)[0]
df  = pipeline.runModelCombinedVola(tickers, 'ipos2-3a', mod, False, loss=lossfn)


loading model from results/ipos2-3a-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
1/1 [==============================] - 0s 9ms/step
loading model from results/ipos2-3a-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
1/1 [==============================] - 0s 43ms/step


1/1 [==============================] - 0s 9ms/step
loading model from results/ipos2-3a-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 10ms/step


In [136]:
addAlloc(df, 0.10, 1)
df.sort_values('Alloc', ascending=False).head(50)


,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
53,RBLX,24.97,0.98,352.83,432.57,45.70,41.720001,-0.09,0.70,202.82,342.80,59.200001,0.30,0.68,347.66,421.32,41.110001,-0.10,9.577778
11,CRWD,36.38,0.96,1567.78,1552.88,135.77,104.339996,-0.23,0.82,1965.61,1514.46,145.360001,0.07,0.72,1303.01,1513.10,113.980003,-0.16,9.426087
19,ESTC,46.04,0.96,950.42,1193.35,59.68,52.490002,-0.12,0.74,779.86,948.24,69.260002,0.16,0.63,679.30,682.26,54.500000,-0.09,9.266667
32,LYFT,9.30,0.95,333.27,497.78,10.17,11.960000,0.18,0.77,323.40,359.36,12.410000,0.22,0.73,171.21,513.00,10.200000,0.00,9.222222
44,OSTK,4.17,0.94,883.33,667.38,18.97,23.830000,0.26,0.66,795.37,676.27,23.540001,0.24,0.68,425.49,322.54,14.460000,-0.24,9.169231
58,STEM,5.30,0.94,135.10,234.45,4.69,5.910000,0.26,0.67,188.30,159.56,6.040000,0.29,0.78,152.12,253.31,5.200000,0.11,9.169231
61,TEAM,111.74,0.95,1941.20,2227.09,165.53,140.309998,-0.15,0.74,2318.79,1733.89,192.720001,0.16,0.72,1714.98,1952.42,137.869995,-0.17,9.166667
46,PAYO,3.62,0.95,37.18,52.28,5.62,6.420000,0.14,0.73,37.65,49.07,7.160000,0.27,0.61,8.60,23.55,5.700000,0.01,9.142857
14,DDOG,29.97,0.95,1002.33,894.26,67.32,58.349998,-0.13,0.76,941.19,771.73,72.889999,0.08,0.74,969.39,1009.79,55.099998,-0.18,9.115385
57,SPOT,75.30,0.94,1857.76,2195.69,134.74,106.779999,-0.21,0.73,1583.12,2089.57,160.470001,0.19,0.72,1703.81,1910.77,127.680000,-0.05,9.114286


In [60]:
import pandas as pd
df1 = pipeline.fetch_data('ABNB')
#df1.sort_index(inplace=True)
df1 = df1.round(3)

df2 = pd.read_csv('data/ABNB_2023-04-15.csv', index_col=0)
df2.index =  pd.to_datetime(df2.index)
# df2 = df2.round(3)
#data2 = data.tail(1000)
#pd.concat([df1,df2]).drop_duplicates(keep=False)
#df1.compare(df2)
#df2.index.dtype
df1

,open,high,low,close,adjclose,volume,ticker
2021-01-08,0.600,0.600,0.470,0.575,0.575,77600,ILAL
2021-01-11,0.575,0.630,0.510,0.620,0.620,16200,ILAL
2021-01-12,0.600,0.600,0.513,0.520,0.520,20700,ILAL
2021-01-13,0.610,0.610,0.502,0.600,0.600,35200,ILAL
2021-01-14,0.600,0.625,0.550,0.625,0.625,77200,ILAL
...,...,...,...,...,...,...,...
2023-04-10,0.105,0.105,0.072,0.105,0.105,38800,ILAL
2023-04-11,0.105,0.105,0.105,0.105,0.105,0,ILAL
2023-04-12,0.080,0.103,0.080,0.103,0.103,1000,ILAL
2023-04-13,0.098,0.148,0.098,0.148,0.148,319700,ILAL


In [ ]:
display(df.sort_values('Gain', ascending=False))
display(df[["Buy", "Sell", "Total"]].sum()/len(df))


In [ ]:
results['ABNB'].pdata.__dict__.keys()

In [ ]:
findata.EPOCHS=10
tickers1 = ['ARKK', 'ARKW', 'DAPP', 'DTEC', 'EEM', 'FPX',
            'GBTC', 'GLD',
            'ICLN', 'IJR', 'IPO', 'IPOS', 'IWM',
            'JETS', 'KEMQ',
            'MGK', 'MGV', 'MOAT', 'MTUM',
            'QQQ', 'SLV','SMH', 'SMOG',
            'SPY', 'VNQ', 'VT', 'VTI', 'WDIV',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME' ]
mod = pipeline.RateReturnOnly(pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap()]))
df1 = pipeline.runModelCombined(tickers1, 'etf2b', mod, True)

In [ ]:
df1.sort_values('Gain', ascending=False)

In [ ]:
findata.EPOCHS=200
tickers3 = [
           'AHT', 'AMSC', 'ASTR', 'ATOM',
           'BKKT', 'BGFV', 'BGSF', 'CBNT', 'CLOV', 
           'DNMR', 'ERJ', 'EVGO',
           'FSLY',  'FTCH', 'GOGO', 'HIVE', 
           'ILAL', 'INLX', 'JMIA', 'JOBY',  'KULR', 'MTTR',
           'MYTE', 'NEPH', 'ONDS', 'MQ',
           'PETS', 'PTON', 'SFIX', 'SFT', 'STNE', 
           'ULH', 'VRAR', 'WISH']
lossfn = "huber_loss"
# lossfn = "mean_squared_error"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df3, results = pipeline.runModelCombined(tickers3, 'vols', mod, False, loss=lossfn)

In [ ]:
df3

In [ ]:
import pandas as pd
seq = list()
for ticker, result in results.items():
    tempdf = result.final_df
    tempdf['predicted_rate'] = (tempdf['adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    tempdf['true_rate'] = (tempdf['true_adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    seq.append(tempdf[['ticker', 'date', "predicted_rate", 'true_rate']])

detailstat = pd.concat(seq)



In [ ]:
normalstat = detailstat[detailstat['predicted_rate'].between(-2,2) & detailstat['true_rate'].between(-2,2)]
largestat = normalstat[~ normalstat['predicted_rate'].between(-0.25,0.25) & ~ normalstat['true_rate'].between(-0.25,0.25)]
largestat.sample(10)

In [ ]:
ax = largestat.plot.scatter(x='predicted_rate', y='true_rate',c='DarkBlue', figsize=(10,10))
# ax.set_yscale('log')
# ax.set_xscale('log')

In [ ]:
# import matplotlib.pyplot as plt

# plt.hist([normalstat['predicted_rate'],normalstat['predicted_rate']],
#          bins=100, range=(-1,1), color = ['r','g'])

corr = largestat[["predicted_rate", "true_rate"]].corr()

corr.style.background_gradient(cmap='coolwarm')

In [ ]:

sns.regplot(x=largestat["predicted_rate"], y=largestat["true_rate"])


In [ ]:
sns.heatmap(normalstat[["predicted_rate", "true_rate"]].corr(), annot = True, fmt='.2g',cmap= 'coolwarm')

In [ ]:
!rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 

sending incremental file list
ipos2-3a-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
ipos2-3a-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
ipos2-3a-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 9,640,232 bytes  received 73 bytes  6,426,870.00 bytes/sec
total size is 9,637,296  speedup is 1.00
